In [1]:
pip install pymoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210082 sha256=07250b88cf2e278fe0d5f8f5b1b7dcbbdb4c4c0f52e7237adb39a037d153efc8
  Stored in directory: /root/.cache/pip/wheels/01/e1/49/37e6bde9886439057450c494a79b0bef8bbe897a54aebfc757
Successfully built grapheme


In [7]:
!git clone https://github.com/KadircanKara/SAR.git
!mv SAR/* SAR/.* . 2>/dev/null
!rm -rf SAR

In [4]:
from PathInput import *
from PathUnitTest import PathUnitTest

### Single Visit

In [ ]:
test = PathUnitTest(scenario=single_visit_setup_scenarios)
test(save_results=True, animation=False)

### Two Visits

In [ ]:
test = PathUnitTest(scenario=two_visits_setup_scenarios)
test(save_results=True, animation=False)

### Three Visits

In [ ]:
test = PathUnitTest(scenario=three_visits_setup_scenarios)
test(save_results=True, animation=False)